In [129]:
# Carregando bibliotecas
import duckdb
import pandas as pd

In [130]:
# Cria uma conexão com o DuckDB e carrega a tabela
conn = duckdb.connect('data/dados_sinan.db')

# Testando Conexão
print(conn.execute("SELECT * FROM dengue.dengue_pe_fat LIMIT 5").fetchdf())

  TP_NOT ID_AGRAVO  DT_NOTIFIC SEM_NOT NU_ANO SG_UF_NOT ID_MUNICIP ID_REGIONA  \
0      2       A90  2025-01-30  202505   2025        26     260720       1497   
1      2       A90  2025-01-11  202502   2025        26     261110       1503   
2      2       A90  2025-01-06  202502   2025        26     261110       1503   
3      2       A90  2025-02-20  202508   2025        26     261110       1503   
4      2       A90  2025-02-06  202506   2025        26     261110       1503   

  ID_UNIDADE  DT_SIN_PRI  ... EVIDENCIA PLAQ_MENOR CON_FHD COMPLICA  \
0    7360339  2025-01-29  ...      <NA>       <NA>    <NA>     <NA>   
1    9698469  2025-01-06  ...      <NA>       <NA>    <NA>     <NA>   
2    9019650  2025-01-03  ...      <NA>       <NA>    <NA>     <NA>   
3    2429853  2025-02-19  ...      <NA>       <NA>    <NA>     <NA>   
4    2430711  2025-02-04  ...      <NA>       <NA>    <NA>     <NA>   

  TP_SISTEMA NDUPLIC_N   DT_DIGITA CS_FLXRET FLXRECEBI MIGRADO_W  
0          2      N

## 🔎 Primeira Consulta – Casos com Comorbidades Graves

Esta consulta seleciona apenas os casos de pacientes **hospitalizados** ou que apresentam **comorbidades** relevantes (como **diabetes**, **hipertensão** ou **hepatopatias**) ou que manifestaram **sinais de gravidade**. 

O objetivo é permitir que o time de análise concentre-se nas **populações de risco** e na **demanda por leitos hospitalares**, oferecendo subsídios para decisões clínicas e estratégicas.


In [131]:
print(conn.execute("""
SELECT 
    DT_NOTIFIC,
    HOSPITALIZ,
    CS_SEXO,
    gest.NOME_GESTANT AS GESTANTE,

    -- RESIDÊNCIA
    mun_res.NOME_MUNICIP AS municipio_residencia,
    uf_res.UF_NOME AS uf_residencia,

    -- NOTIFICAÇÃO
    mun_not.NOME_MUNICIP AS municipio_notificacao,
    uf_not.UF_NOME AS uf_notificacao,

    -- SINAIS/SINTOMAS
    FEBRE,
    MIALGIA,
    CEFALEIA,
    EXANTEMA,
    VOMITO,
    NAUSEA,
    DOR_COSTAS,
    CONJUNTVIT,
    ARTRITE,
    ARTRALGIA,
    PETEQUIA_N,
    LEUCOPENIA,
    LACO,
    DOR_RETRO,

    -- COMORBIDADES
    DIABETES,
    HEMATOLOG,
    HEPATOPAT,
    RENAL,
    HIPERTENSA,
    ACIDO_PEPT,
    AUTO_IMUNE,

    -- SINAIS DE ALERTA
    ALRM_HIPOT,
    ALRM_PLAQ,
    ALRM_VOM,
    ALRM_SANG,
    ALRM_HEMAT,
    ALRM_ABDOM,
    ALRM_LETAR,
    ALRM_HEPAT,
    ALRM_LIQ,

    -- SINAIS DE GRAVIDADE
    GRAV_PULSO,
    GRAV_CONV,
    GRAV_ENCH,
    GRAV_INSUF,
    GRAV_TAQUI,
    GRAV_EXTRE,
    GRAV_HIPOT,
    GRAV_HEMAT,
    GRAV_MELEN,
    GRAV_METRO,
    GRAV_SANG,
    GRAV_AST,
    GRAV_MIOC,
    GRAV_CONSC,
    GRAV_ORGAO,

    -- MANIFESTAÇÕES HEMORRÁGICAS
    MANI_HEMOR,
    EPISTAXE,
    GENGIVO,
    METRO,
    PETEQUIAS,
    HEMATURA,
    SANGRAM,

    -- OUTROS
    LACO_N,
    PLASMATICO,
    EVIDENCIA,
    PLAQ_MENOR,
    CON_FHD,
    COMPLICA

FROM dengue.dengue_pe_fat AS casos

-- JOIN MUNICÍPIO DE RESIDÊNCIA
LEFT JOIN dengue.dim_municipios AS mun_res
    ON casos.ID_MN_RESI = mun_res.ID_MUNICIP

-- JOIN UF DE RESIDÊNCIA
LEFT JOIN dengue.dim_ufs AS uf_res
    ON casos.SG_UF = uf_res.SG_UF

-- JOIN MUNICÍPIO DE NOTIFICAÇÃO
LEFT JOIN dengue.dim_municipios AS mun_not
    ON casos.ID_MUNICIP = mun_not.ID_MUNICIP

-- JOIN UF DE NOTIFICAÇÃO
LEFT JOIN dengue.dim_ufs AS uf_not
    ON casos.SG_UF = uf_not.SG_UF

-- JOIN TEMPO GESTANTE
LEFT JOIN dengue.dim_gestante AS gest
    ON casos.CS_GESTANT = gest.CS_GESTANT

WHERE
    (DIABETES = TRUE OR HIPERTENSA = TRUE OR HEPATOPAT = TRUE)
    OR (GRAV_PULSO = TRUE OR
    GRAV_CONV = TRUE OR
    GRAV_ENCH = TRUE OR
    GRAV_INSUF = TRUE OR
    GRAV_TAQUI = TRUE OR
    GRAV_EXTRE = TRUE OR
    GRAV_HIPOT = TRUE OR
    GRAV_HEMAT = TRUE OR
    GRAV_MELEN = TRUE OR
    GRAV_METRO = TRUE OR
    GRAV_SANG = TRUE OR
    GRAV_AST = TRUE OR
    GRAV_MIOC = TRUE OR
    GRAV_CONSC = TRUE OR
    GRAV_ORGAO = TRUE)
    OR casos.CS_GESTANT IN ('1','2','3','4')
    OR hospitaliz = TRUE

ORDER BY DT_NOTIFIC DESC;
""").fetchdf())

     DT_NOTIFIC  HOSPITALIZ CS_SEXO       GESTANTE municipio_residencia  \
0    2025-05-03        True       F  Não se aplica               Recife   
1    2025-05-02        True       F  Não se aplica            Salgueiro   
2    2025-05-02        True       F   3º Trimestre               Olinda   
3    2025-04-30       False       F       Ignorado               Recife   
4    2025-04-30        True       F   3º Trimestre             Paulista   
..          ...         ...     ...            ...                  ...   
818  2025-01-02        True       F  Não se aplica             Paulista   
819  2025-01-02        True       M  Não se aplica               Olinda   
820  2025-01-01        <NA>       M  Não se aplica              Jataúba   
821  2025-01-01        <NA>       M  Não se aplica              Jataúba   
822  2024-12-30        True       M  Não se aplica               Recife   

    uf_residencia municipio_notificacao uf_notificacao  FEBRE  MIALGIA  ...  \
0      Pernambuco   

In [ ]:
print(conn.execute("""
SELECT 
    DT_NOTIFIC,
    HOSPITALIZ,
    CASE WHEN classi_fin = '1' THEN 'CONFIRMADO' ELSE 'DESCARTADO' END AS CLASSIFICACAO_FINAL,
    CS_SEXO,
    CS_RACA,
    esc.NOME_ESCOL AS ESCOLARIDADE,

    -- RESIDÊNCIA
    mun_res.NOME_MUNICIP AS municipio_residencia,
    uf_res.UF_NOME AS uf_residencia,

    -- NOTIFICAÇÃO
    mun_not.NOME_MUNICIP AS municipio_notificacao,
    uf_not.UF_NOME AS uf_notificacao,
    
    -- SINAIS/SINTOMAS
    FEBRE,
    MIALGIA,
    CEFALEIA,
    EXANTEMA,
    VOMITO,
    NAUSEA,
    DOR_COSTAS,
    CONJUNTVIT,
    ARTRITE,
    ARTRALGIA,
    PETEQUIA_N,
    LEUCOPENIA,
    LACO,
    DOR_RETRO
                   
FROM dengue.dengue_pe_fat AS casos

-- JOIN MUNICÍPIO DE RESIDÊNCIA
LEFT JOIN dengue.dim_municipios AS mun_res
    ON casos.ID_MN_RESI = mun_res.ID_MUNICIP

-- JOIN UF DE RESIDÊNCIA
LEFT JOIN dengue.dim_ufs AS uf_res
    ON casos.SG_UF = uf_res.SG_UF

-- JOIN MUNICÍPIO DE NOTIFICAÇÃO
LEFT JOIN dengue.dim_municipios AS mun_not
    ON casos.ID_MUNICIP = mun_not.ID_MUNICIP

-- JOIN UF DE NOTIFICAÇÃO
LEFT JOIN dengue.dim_ufs AS uf_not
    ON casos.SG_UF = uf_not.SG_UF

LEFT JOIN dengue.dim_escolaridade AS esc
    ON casos.CS_ESCOL_N = esc.CS_ESCOL_N

WHERE classi_fin IN ('1', '2')

ORDER BY DT_NOTIFIC DESC;
""").fetchdf())

BinderException: Binder Error: Referenced table "est" not found!
Candidate tables: "esc", "casos"

LINE 8:     est.NOME_ESCOL AS ESCOLARIDADE,
            ^

In [ ]:
conn.close()